In [1]:

import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time


In [5]:

model = load_model("model.keras")
print("model loaded successfully!")

model loaded successfully!


In [6]:

# Open camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Camera not found!")
    exit()

print("Show digits to the camera. Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get region of interest (center box)
    h, w, _ = frame.shape
    box_w, box_h = int(w*0.6), int(h*0.6)
    x1, y1 = w//2 - box_w//2, h//2 - box_h//2
    x2, y2 = x1 + box_w, y1 + box_h
    roi = frame[y1:y2, x1:x2]

    # Preprocess for model
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28)) / 255.0
    input_data = np.expand_dims(resized, axis=(0, -1))

    # Predict digit
    preds = model.predict(input_data, verbose=0)
    digit = np.argmax(preds)
    confidence = np.max(preds)

    # Show results
    if confidence > 0.7:
        text = f"Digit: {digit} ({confidence:.2f})"
        color = (0, 255, 0)
    else:
        text = "Searching for digit..."
        color = (0, 0, 255)

    cv2.putText(frame, text, (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)

    cv2.imshow("Digit Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

  

Show digits to the camera. Press 'q' to quit
